In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import nltk.data
from nltk.tokenize import BlanklineTokenizer
import sys
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
# test = open("/content/drive/MyDrive/Hybridization/input/en-hi-wsd-3L/output_en-hi_WSD-500_intrain.txt", "r")
# reference = open("/content/drive/MyDrive/Hybridization/output/en-hi/ref_en-hi-500-intrain.txt", "r")
# reference = open("/content/drive/MyDrive/MTE_Files/METEOR/parallel-n/english.txt","r")
# test = open("/content/drive/MyDrive/MTE_Files/METEOR/parallel-n/hindi.txt","r")

# reference = open("/content/drive/MyDrive/MTdata/references(human).txt")
# test = open("/content/drive/MyDrive/MTdata/hypothesis(machine).txt")

# reference = open("/content/drive/MyDrive/MTdata/benRef.txt")
# test = open("/content/drive/MyDrive/MTdata/benHyp.txt")

reference = open("/content/drive/MyDrive/MTdata/Telugu/ref.txt")
test = open("/content/drive/MyDrive/MTdata/Telugu/yandex_hypo.txt")

In [5]:
list1 = []
list2 = []
for line in reference:
    #print(line)
    list1.append(BlanklineTokenizer().tokenize(line))
for line in test:
    #print(line)
    list2.append(BlanklineTokenizer().tokenize(line))
reference.close()
test.close()

In [6]:
s1=[]
s2=[]
for ele in list1:
    for elem in ele:
        s1.append(" ".join([str(elem) for elem in ele]))
for ele in list2:
    for elem in ele:
        s2.append(" ".join([str(elem) for elem in ele]))

In [7]:
le=len(s2)
te = len(s1)
print(le)
print(te)

27
27


In [8]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from itertools import chain, product


def _generate_enums(hypothesis, reference, preprocess=str.lower):
    hypothesis_list = list(enumerate(preprocess(hypothesis).split()))
    reference_list = list(enumerate(preprocess(reference).split()))
    return hypothesis_list, reference_list

def _enum_stem_match(enum_hypothesis_list, enum_reference_list, stemmer = PorterStemmer()):
    stemmed_enum_list1 = [(word_pair[0],stemmer.stem(word_pair[1])) \
                          for word_pair in enum_hypothesis_list]

    stemmed_enum_list2 = [(word_pair[0],stemmer.stem(word_pair[1])) \
                          for word_pair in enum_reference_list]

    word_match, enum_unmat_hypo_list, enum_unmat_ref_list = \
                    _match_enums(stemmed_enum_list1, stemmed_enum_list2)

    enum_unmat_hypo_list = list(zip(*enum_unmat_hypo_list)) if len(enum_unmat_hypo_list)>0 else []

    enum_unmat_ref_list = list(zip(*enum_unmat_ref_list)) if len(enum_unmat_ref_list)>0 else []

    enum_hypothesis_list = list(filter(lambda x:x[0] not in enum_unmat_hypo_list,  
                                       enum_hypothesis_list))

    enum_reference_list = list(filter(lambda x:x[0] not in enum_unmat_ref_list, 
                                      enum_reference_list))

    return word_match, enum_hypothesis_list, enum_reference_list




def _match_enums(enum_hypothesis_list, enum_reference_list):
    word_match = []
    for i in range(len(enum_hypothesis_list))[::-1]:
        for j in range(len(enum_reference_list))[::-1]:
            if enum_hypothesis_list[i][1] == enum_reference_list[j][1]:
                word_match.append((enum_hypothesis_list[i][0],enum_reference_list[j][0]))
                (enum_hypothesis_list.pop(i)[1],enum_reference_list.pop(j)[1])
                break
    return word_match, enum_hypothesis_list, enum_reference_list


def _enum_wordnetsyn_match(enum_hypothesis_list, enum_reference_list, wordnet = wordnet):
    word_match = []
    for i in range(len(enum_hypothesis_list))[::-1]:
        hypothesis_syns = set(chain(*[[lemma.name() for lemma in synset.lemmas() 
                                        if lemma.name().find('_')<0] 
                                       for synset in \
                                           wordnet.synsets(
                                               enum_hypothesis_list[i][1])]
                                               )).union({enum_hypothesis_list[i][1]})
        for j in range(len(enum_reference_list))[::-1]:
            if enum_reference_list[j][1] in hypothesis_syns:
                word_match.append((enum_hypothesis_list[i][0],enum_reference_list[j][0]))
                enum_hypothesis_list.pop(i),enum_reference_list.pop(j)
                break
    return word_match, enum_hypothesis_list, enum_reference_list


def _count_chunks(matches):
    i=0
    chunks = 1
    while(i<len(matches)-1):
        if (matches[i+1][0]==matches[i][0]+1) and (matches[i+1][1]==matches[i][1]+1):
            i+=1
            continue
        i+=1
        chunks += 1
    return chunks






def _enum_allign_words(enum_hypothesis_list, enum_reference_list, 
                       stemmer=PorterStemmer(), wordnet = wordnet):
    exact_matches, enum_hypothesis_list, enum_reference_list = \
        _match_enums(enum_hypothesis_list, enum_reference_list)

    stem_matches, enum_hypothesis_list, enum_reference_list = \
        _enum_stem_match(enum_hypothesis_list, enum_reference_list,
                         stemmer = stemmer)

    wns_matches, enum_hypothesis_list, enum_reference_list = \
        _enum_wordnetsyn_match(enum_hypothesis_list, enum_reference_list, 
                               wordnet = wordnet)

    return (sorted(exact_matches + stem_matches + wns_matches, key=lambda wordpair:wordpair[0]),
            enum_hypothesis_list, enum_reference_list)




def single_meteor_score(reference, 
                        hypothesis, 
                        preprocess = str.lower, 
                        stemmer = PorterStemmer(), 
                        wordnet = wordnet, 
                        alpha=0.9, 
                        beta=3, 
                        gamma=0.5):
    enum_hypothesis, enum_reference = _generate_enums(hypothesis, 
                                                      reference, 
                                                      preprocess = preprocess)
    #print(enum_hypothesis, enum_reference)
    
    translation_length = len(enum_hypothesis)
    reference_length = len(enum_reference)
    matches, _, _ = _enum_allign_words(enum_hypothesis, enum_reference)
    
    #print(matches, _, _)
    matches_count = len(matches)
    try:
        precision = float(matches_count)/translation_length
        recall = float(matches_count)/reference_length
        fmean = (precision*recall)/(alpha*precision+(1-alpha)*recall)
        chunk_count = float(_count_chunks(matches))
        frag_frac = chunk_count/matches_count
    except ZeroDivisionError:
        return 0.0        
    penalty = gamma*frag_frac**beta
    return (1-penalty)*fmean


In [9]:
lop=0
scor=[]
coun=0
for lop in range(le):
    scor.append(single_meteor_score(s1[lop],s2[lop]))

In [10]:
f1 = open('/content/meteor_blank_score_ur1.txt', "w")
lope=0
su=0
for lope in range(le):
    print(f"{lope+1}.   {scor[lope]}")
    su=su+scor[lope]
    f1.write(str(scor[lope])+'\n') #Sentence_score
f1.close()
su=su/le
su #Corpus_score

1.   0.3896604938271605
2.   0.33848417954378224
3.   0.12139423076923078
4.   0.755560651226783
5.   0.5033238366571701
6.   0.14705882352941174
7.   0.4445422535211268
8.   0.04132231404958677
9.   0.25833333333333336
10.   0.3468406593406593
11.   0.40812807881773394
12.   0.3647058823529412
13.   0.6916666666666668
14.   0.5
15.   0.20444444444444443
16.   0.5506172839506173
17.   0.08971291866028708
18.   0.47700840937555916
19.   0.04201680672268908
20.   0.5367768595041322
21.   0.12820512820512822
22.   0.2508169934640523
23.   0.36666666666666664
24.   0.5575
25.   0.4310344827586206
26.   0.43945312499999994
27.   0.42613636363636365


0.3633855883712647

In [4]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
f5 = open('/content/meteor_blank_score_ur1.txt', "a+")
with open(test, "r") as f1, open(reference, "r") as f2:
  lines1 = [line.rstrip('\n') for line in f1]
  print(len(lines1))
  lines2 = [line.rstrip('\n') for line in f2]
  print(len(lines2))
  for l1,l2 in zip(lines1,lines2):
    tokens_ref=word_tokenize(l2)
    #print(tokens_ref)
    tokens_translate=word_tokenize(l1)
    #print(tokens_translate)
    score = nltk.translate.meteor_score.single_meteor_score(tokens_ref, tokens_translate)
    f5.write(str(score)+'\n')
f5.close()

TypeError: ignored

In [13]:
from sklearn import preprocessing
import numpy as np

In [14]:
file = "/content/drive/MyDrive/MTdata/Telugu/yandex_metrics/meteor/meteor_blank_score_ur1-yt.txt"
with open(file, 'r') as f:
  lines = [line.rstrip('\n') for line in f]
  print(lines)
  lst_int = [float(i) for i in lines]
  print(lst_int)
  m_score = list(lst_int)
  # normalized_arr = preprocessing.normalize([lst_int])
  # print(type(normalized_arr))
  # print(type(list(normalized_arr[0])))
  # print(type(normalized_arr[0]))
  # m_score = list(normalized_arr[0])
  # print(m_score)
  # up_score = []
  # for i in m_score:
  #   up_score.append(i)
  # print(up_score)
  

['0.3896604938271605', '0.33848417954378224', '0.12139423076923078', '0.755560651226783', '0.5033238366571701', '0.14705882352941174', '0.4445422535211268', '0.04132231404958677', '0.25833333333333336', '0.3468406593406593', '0.40812807881773394', '0.3647058823529412', '0.6916666666666668', '0.5', '0.20444444444444443', '0.5506172839506173', '0.08971291866028708', '0.47700840937555916', '0.04201680672268908', '0.5367768595041322', '0.12820512820512822', '0.2508169934640523', '0.36666666666666664', '0.5575', '0.4310344827586206', '0.43945312499999994', '0.42613636363636365']
[0.3896604938271605, 0.33848417954378224, 0.12139423076923078, 0.755560651226783, 0.5033238366571701, 0.14705882352941174, 0.4445422535211268, 0.04132231404958677, 0.25833333333333336, 0.3468406593406593, 0.40812807881773394, 0.3647058823529412, 0.6916666666666668, 0.5, 0.20444444444444443, 0.5506172839506173, 0.08971291866028708, 0.47700840937555916, 0.04201680672268908, 0.5367768595041322, 0.12820512820512822, 0.2

In [16]:
file = "/content/drive/MyDrive/MTdata/Telugu/yandex_score.txt"
with open(file, 'r') as f:
  lines = [line.rstrip('\n') for line in f]
  lst_dble = [float(i) for i in lines]
  print((lst_dble))

[0.5, 0.8, 0.6, 0.9, 0.8, 0.5, 0.6, 0.4, 0.6, 0.7, 0.6, 0.7, 0.8, 0.5, 0.6, 0.8, 0.7, 0.7, 0.8, 0.3, 0.5, 0.6, 0.6, 0.5, 0.4, 0.5, 0.6]


In [17]:
import pandas as pd
from scipy.stats import pearsonr

corr, _ = pearsonr(m_score, lst_dble)
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.226
